In [1]:
%rm -rf LLaMA-Factory
!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install .

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 8222, done.
remote: Counting objects: 100% (1999/1999), done.
remote: Compressing objects: 100% (280/280), done.
remote: Total 8222 (delta 1857), reused 1728 (delta 1719), pack-reused 6223
Receiving objects: 100% (8222/8222), 205.68 MiB | 19.69 MiB/s, done.
Resolving deltas: 100% (6043/6043), done.
Updating files: 100% (184/184), done.
/content/LLaMA-Factory
assets/             Dockerfile   LICENSE         README.md         scripts/  tests/
data/               evaluation/  Makefile        README_zh.md      setup.py
docker-compose.yml  examples/    pyproject.toml  requirements.txt  src/
Processing /content/LLaMA-Factory
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/

In [2]:
from llmtuner import run_exp, ChatModel, export_model


import torch
torch.cuda.is_available()

True

We are using Qwen, small but an efficient model.
Feel free to explore other Qwen models at: https://huggingface.co/Qwen
Also you can explore other supported architectures in LLaMA Factory official repo

Here we are using identity,alpaca_gpt4_en,alpaca_gpt4_zh datasets to fine-tune the model, you can explore other datasets in hugging face according to the need.

In [3]:
# Here we train the model and export it to a directory called "E2E_Qwen1.5-0.5B-ChatBot"
run_exp(dict(
  stage="sft",
  do_train=True,
  model_name_or_path="Qwen/Qwen1.5-0.5B-Chat",
  dataset="identity,alpaca_gpt4_en,alpaca_gpt4_zh",
  template="qwen",
  finetuning_type="lora",
  lora_target="all",
  output_dir="E2E_Qwen1.5-0.5B-ChatBot",
  per_device_train_batch_size=4,
  gradient_accumulation_steps=4,
  lr_scheduler_type="cosine",
  logging_steps=10,
  save_steps=100,
  learning_rate=1e-4,
  num_train_epochs=5.0,
  max_samples=500,
  max_grad_norm=1.0,
  fp16=True,
))

03/16/2024 06:37:25 - INFO - llmtuner.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16


INFO:llmtuner.hparams.parser:Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

03/16/2024 06:38:00 - INFO - llmtuner.model.loader - trainable params: 3784704 || all params: 467772416 || trainable%: 0.8091


INFO:llmtuner.model.loader:trainable params: 3784704 || all params: 467772416 || trainable%: 0.8091
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
[INFO|trainer.py:601] 2024-03-16 06:38:00,788 >> Using auto half precision backend
[INFO|trainer.py:1812] 2024-03-16 06:38:01,200 >> ***** Running training *****
[INFO|trainer.py:1813] 2024-03-16 06:38:01,202 >>   Num examples = 1,091
[INFO|trainer.py:1814] 2024-03-16 06:38:01,205 >>   Num Epochs = 5
[INFO|trainer.py:1815] 2024-03-16 06:38:01,206 >>   Instantaneous batch size per device = 4
[INF

Step,Training Loss
10,1.894700
20,1.747600
30,1.691200
40,1.739900
50,1.748100
60,1.707600
70,1.620200
80,1.694400
90,1.643400
100,1.590500


[INFO|trainer.py:3067] 2024-03-16 06:40:59,645 >> Saving model checkpoint to E2E_Qwen1.5-0.5B-ChatBot/tmp-checkpoint-100
[INFO|configuration_utils.py:728] 2024-03-16 06:41:00,068 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-0.5B-Chat/snapshots/6c705984bb8b5591dd4e1a9e66e1a127965fd08d/config.json
[INFO|configuration_utils.py:791] 2024-03-16 06:41:00,070 >> Model config Qwen2Config {
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 2816,
  "max_position_embeddings": 32768,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_key_value_heads": 16,
  "rms_norm_eps": 1e-06,
  "rope_theta": 1000000.0,
  "sliding_window": 32768,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "tran

***** train metrics *****
  epoch                    =       4.98
  total_flos               =  2579190GF
  train_loss               =     1.5998
  train_runtime            = 0:10:06.93
  train_samples_per_second =      8.988
  train_steps_per_second   =       0.56


Note that we have a folder created inside the LLaMA-Factory with the trained model checkpoints. We can export the fine tuned model to "E2E-ChatBot" directory using the below code

In [4]:
export_model(dict(
  model_name_or_path="Qwen/Qwen1.5-0.5B-Chat",
  adapter_name_or_path="E2E_Qwen1.5-0.5B-ChatBot",
  finetuning_type="lora",
  template="qwen",
  export_dir="E2E-ChatBot",
))

[INFO|tokenization_utils_base.py:2046] 2024-03-16 06:48:18,688 >> loading file vocab.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-0.5B-Chat/snapshots/6c705984bb8b5591dd4e1a9e66e1a127965fd08d/vocab.json
[INFO|tokenization_utils_base.py:2046] 2024-03-16 06:48:18,689 >> loading file merges.txt from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-0.5B-Chat/snapshots/6c705984bb8b5591dd4e1a9e66e1a127965fd08d/merges.txt
[INFO|tokenization_utils_base.py:2046] 2024-03-16 06:48:18,692 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2046] 2024-03-16 06:48:18,693 >> loading file special_tokens_map.json from cache at None
[INFO|tokenization_utils_base.py:2046] 2024-03-16 06:48:18,694 >> loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-0.5B-Chat/snapshots/6c705984bb8b5591dd4e1a9e66e1a127965fd08d/tokenizer_config.json
[INFO|tokenization_utils_base.py:2046] 2024-03-16 06:48:18,

03/16/2024 06:48:19 - INFO - llmtuner.model.patcher - Using KV cache for faster generation.


INFO:llmtuner.model.patcher:Using KV cache for faster generation.
[INFO|modeling_utils.py:3257] 2024-03-16 06:48:19,092 >> loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-0.5B-Chat/snapshots/6c705984bb8b5591dd4e1a9e66e1a127965fd08d/model.safetensors
[INFO|modeling_utils.py:1400] 2024-03-16 06:48:19,106 >> Instantiating Qwen2ForCausalLM model under default dtype torch.float16.
[INFO|configuration_utils.py:845] 2024-03-16 06:48:19,112 >> Generate config GenerationConfig {
  "bos_token_id": 151643,
  "eos_token_id": 151645
}

[INFO|modeling_utils.py:3992] 2024-03-16 06:48:28,124 >> All model checkpoint weights were used when initializing Qwen2ForCausalLM.

[INFO|modeling_utils.py:4000] 2024-03-16 06:48:28,126 >> All the weights of Qwen2ForCausalLM were initialized from the model checkpoint at Qwen/Qwen1.5-0.5B-Chat.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Qwen2ForCausalLM fo

03/16/2024 06:48:29 - INFO - llmtuner.model.adapter - Fine-tuning method: LoRA


INFO:llmtuner.model.adapter:Fine-tuning method: LoRA


...

INFO:llmtuner.model.adapter:Loaded adapter(s): E2E_Qwen1.5-0.5B-ChatBot


03/16/2024 06:48:29 - INFO - llmtuner.model.loader - all params: 463987712


INFO:llmtuner.model.loader:all params: 463987712


03/16/2024 06:48:29 - INFO - llmtuner.data.template - Replace eos token: <|im_end|>


INFO:llmtuner.data.template:Replace eos token: <|im_end|>
[INFO|configuration_utils.py:473] 2024-03-16 06:48:29,921 >> Configuration saved in E2E-ChatBot/config.json
[INFO|configuration_utils.py:614] 2024-03-16 06:48:29,924 >> Configuration saved in E2E-ChatBot/generation_config.json
[INFO|modeling_utils.py:2454] 2024-03-16 06:48:37,869 >> Model weights saved in E2E-ChatBot/model.safetensors
[INFO|tokenization_utils_base.py:2459] 2024-03-16 06:48:37,875 >> tokenizer config file saved in E2E-ChatBot/tokenizer_config.json
[INFO|tokenization_utils_base.py:2468] 2024-03-16 06:48:37,878 >> Special tokens file saved in E2E-ChatBot/special_tokens_map.json
[INFO|tokenization_utils_base.py:2519] 2024-03-16 06:48:37,880 >> added tokens file saved in E2E-ChatBot/added_tokens.json


Now Let's play with the trained model

In [6]:
E2E_Chat_Bot = ChatModel(
    dict(
  model_name_or_path="Qwen/Qwen1.5-0.5B-Chat",
  adapter_name_or_path="E2E_Qwen1.5-0.5B-ChatBot",
  finetuning_type="lora",
  template="qwen",
))

[INFO|tokenization_utils_base.py:2046] 2024-03-16 06:52:39,702 >> loading file vocab.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-0.5B-Chat/snapshots/6c705984bb8b5591dd4e1a9e66e1a127965fd08d/vocab.json
[INFO|tokenization_utils_base.py:2046] 2024-03-16 06:52:39,703 >> loading file merges.txt from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-0.5B-Chat/snapshots/6c705984bb8b5591dd4e1a9e66e1a127965fd08d/merges.txt
[INFO|tokenization_utils_base.py:2046] 2024-03-16 06:52:39,705 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2046] 2024-03-16 06:52:39,707 >> loading file special_tokens_map.json from cache at None
[INFO|tokenization_utils_base.py:2046] 2024-03-16 06:52:39,709 >> loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-0.5B-Chat/snapshots/6c705984bb8b5591dd4e1a9e66e1a127965fd08d/tokenizer_config.json
[INFO|tokenization_utils_base.py:2046] 2024-03-16 06:52:39,

03/16/2024 06:52:40 - INFO - llmtuner.model.patcher - Using KV cache for faster generation.


INFO:llmtuner.model.patcher:Using KV cache for faster generation.
[INFO|modeling_utils.py:3257] 2024-03-16 06:52:40,131 >> loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-0.5B-Chat/snapshots/6c705984bb8b5591dd4e1a9e66e1a127965fd08d/model.safetensors
[INFO|modeling_utils.py:1400] 2024-03-16 06:52:40,151 >> Instantiating Qwen2ForCausalLM model under default dtype torch.float16.
[INFO|configuration_utils.py:845] 2024-03-16 06:52:40,156 >> Generate config GenerationConfig {
  "bos_token_id": 151643,
  "eos_token_id": 151645
}

[INFO|modeling_utils.py:3992] 2024-03-16 06:52:48,346 >> All model checkpoint weights were used when initializing Qwen2ForCausalLM.

[INFO|modeling_utils.py:4000] 2024-03-16 06:52:48,350 >> All the weights of Qwen2ForCausalLM were initialized from the model checkpoint at Qwen/Qwen1.5-0.5B-Chat.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Qwen2ForCausalLM fo

03/16/2024 06:52:49 - INFO - llmtuner.model.adapter - Fine-tuning method: LoRA


INFO:llmtuner.model.adapter:Fine-tuning method: LoRA


...

INFO:llmtuner.model.adapter:Loaded adapter(s): E2E_Qwen1.5-0.5B-ChatBot


03/16/2024 06:52:49 - INFO - llmtuner.model.loader - all params: 463987712


INFO:llmtuner.model.loader:all params: 463987712


03/16/2024 06:52:50 - INFO - llmtuner.data.template - Replace eos token: <|im_end|>


INFO:llmtuner.data.template:Replace eos token: <|im_end|>


In [19]:
messages = []
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break
  if query.strip() == "clear":
    messages = []
    continue

  messages.append({"role": "user", "content": query})
  print("E2E_Chat_Bot: ", end="", flush=True)
  response = ""
  for new_text in E2E_Chat_Bot.stream_chat(messages):
    response += new_text
  for r in response.split('.'):
    print(r.strip(), flush=True)
  print()
  messages.append({"role": "assistant", "content": response})


User: Hi
E2E_Chat_Bot: Hello! How can I help you today?


User: Do you know about E2E cloud network?
E2E_Chat_Bot: Yes, I am familiar with E2E Cloud Network
E2E Cloud Network is a technology that allows companies to host their applications and services in the cloud, allowing them to easily access and manage their resources and applications from anywhere in the world
It is a powerful solution that enables organizations to leverage the capabilities of the cloud to improve their operations and growth



User: can it also be used to train and deploy AI models?
E2E_Chat_Bot: Yes, E2E Cloud Network can be used to train and deploy AI models
In fact, it is a popular choice for organizations that want to deploy AI models in the cloud, as it enables them to easily scale and deploy their models to meet the needs of their users
E2E Cloud Network allows for easy deployment of AI models, with features such as containerization, automatic scaling, and automatic deployment, which can help organization